#### Name :  2-2-CkipCutDemo
#### Function :  
Text segmentation demo using Ckip Tagger

#### Data:    
(1) KccDict2020.txt - user  dictionary             
(2) KccStopWord2020.txt -  stop words

## 斷詞套件比較

### 2-2 CKIPtagger
在 2-2 我們先用 CKIPtagger 示範簡單的斷詞效果。</br>

+ 首先需要下載 CKIPtagger 套件 </br>
`pip install ckiptagger`

+ 下載model </br>
透過這個連結下載</br>
http://ckip.iis.sinica.edu.tw/data/ckiptagger/data.zip

+ 接著檢查 tensorflow需要是  >= 1.13.1 而且 <2 </br>
如果沒有的話可以直接下載</br>
`pip install tensorflow==1.13.1`

+ 詳細解說請看</br>
https://medium.com/@br19920702/%E4%B8%AD%E7%A0%94%E9%99%A2%E9%96%8B%E6%BA%90nlp%E5%A5%97%E4%BB%B6-ckiptagger-%E7%B9%81%E4%B8%AD%E4%B8%8D%E7%B5%90%E5%B7%B4-7822fc4efbf

In [2]:
# -*- coding: utf-8 -*-
"""
Created on May 20 2020
@author: johnson
"""

#一開始先載入套件
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import gc
import tensorflow as tf
# import keras

# 定義ckiptagger要用的詞典 KccDict2020.txt
User_Dict = {}
with open("../KCC Data/Dict/KccDict2020.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 150    # 未給定權重者一律賦予預設值 150     
            
dictionary = construct_dictionary(User_Dict) #指定字典

# 載入模型
ws = WS("../KCC Data/data") #斷詞
pos = POS("../KCC Data/data") #詞性標註
ner = NER("../KCC Data/data") #實體辨識

# 載入 StopWord 字典 StopWordTest.txt
stopword_list = []
with open("../KCC Data/Dict/StopWordTest.txt","r", encoding='utf-8-sig') as stopwords:
    for stopword in stopwords:
        stopword_list.append(stopword.strip())
        
        
#待會要斷的字
#第一句 str
str = "小姐，大家好。高雄縣市已經合併，陳市長一直強調「無縫接軌」強調火星人，石化氣爆"
str += "到目前為止，在各方面的交接運作，大致上還算不錯；但是有一小部分，高雄縣、高雄市留下來的法令規範有不一樣的地方，"
str += "教育局長范巽綠：要如何做處理，我相信還有其他的問題，大家要再研議、協商，看要如何修改。高雄縣市合併以後， 長期照護協會要何去何從，我們都知道"

#第二句 str2
str2 = "這些方向應該沒有問題，我們是往非營利幼兒園的模式所以用人等會一 這些方向應該沒有問題，"
str2 += "我們是往非營利幼兒園的模式所以用人等會一 這些方向應該沒有問題，"
str2 += "黃議員石龍,周議員玲妏：我們就往這個方向來邁進，石化氣爆真的不容許再為建設而浪費麼多從中央申請"
    
    
# 開始斷詞
# sentence_segmentation = True,  => To consider delimiters
# segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
# recommend_dictionary = dictionary1, # words in this dictionary are encouraged
# coerce_dictionary = dictionary2, # words in this dictionary are forced

# 用 ws 斷詞，結果存在 strCut
strCut = ws([str], sentence_segmentation=True,
                 segment_delimiter_set = {'"',"\r\n","，","「","」",",","？",'\n', "。", ":", "?", "!", ";", "、"}, #過濾標點符號
                 coerce_dictionary = dictionary) #剛剛上面定義的dictionary

print("------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------")

#把斷詞結果印出來

stayed_lst= [] #存印出來的list的空字串
for word in strCut:
    
    #去除stop word字典裡的字
    if word not in stopword_list: #如果strCut裡面的字不再字典裡
        stayed_lst.append(word) #加到stayed_lst裡面
        print(" / ".join(stayed_lst[0])) #把stayed_lst印出來


------------CKIPTagger Word Segment ws(str,....) + 刪除標點及停用字 ---------------
小姐 / ， / 大家 / 好 / 。 / 高雄縣市 / 已經 / 合併 / ， / 陳 / 市長 / 一直 / 強調 / 「 / 無縫接軌 / 」 / 強調 / 火星人 / ， / 石化氣爆 / 到 / 目前 / 為止 / ， / 在 / 各 / 方面 / 的 / 交接 / 運作 / ， / 大致上 / 還 / 算 / 不錯 / ； / 但是 / 有 / 一 / 小 / 部分 / ， / 高雄縣 / 、 / 高雄市 / 留下來 / 的 / 法令規範 / 有 / 不 / 一樣 / 的 / 地方 / ， / 教育局長 / 范巽綠 / ： / 要 / 如何 / 做 / 處理 / ， / 我 / 相信 / 還 / 有 / 其他 / 的 / 問題 / ， / 大家 / 要 / 再 / 研議 / 、 / 協商 / ， / 看 / 要 / 如何 / 修改 / 。 / 高雄縣市 / 合併 / 以後 / ， /   / 長期照護協會 / 要 / 何去何從 / ， / 我們 / 都 / 知道
